## Auto Shop Group

![](https://gitlab.com/ch-tbz-wb/Stud/CnA/-/raw/main/2_Unterrichtsressourcen/x_gitressourcen/d-eventing.png)

- - -

Die Auto Shop GmbH ist zur Auto Shop Group expandiert und unterhält eine grosse Anzahl von Garagen.

Diese sind mit Touch Screens ausgestattet, wo die Mechaniker/innen Kundenarbeiten und Verbrauchsmaterial direkt abrechnen können.

Die Touch Screen senden die Daten mittels MQTT-Protokolls im CSV-Format. Ein «vor Ort» installiert Kubernetes Cluster wandelt die Daten nach HTTP und JSON und übermittelt diese an die Konzernzentrale.

In der Konzernzentrale steht eine CloudEvents basierende Messaging Lösung (Broker) zur Verfügung. Diese Empfängt die Daten und verteilt diese an die Fachapplikationen.


- - -

Zuerst brauchen wir die IIoT Devices als K8s Resources

In [ ]:
%%bash
! [ -d iiot ] && git clone https://gitlab.com/ch-mc-b/autoshop-ms/infra/iiot.git 
cd iiot
kubectl apply -f crd

**AWS**

Für AWS muss der MQTT Broker URL, von intern `10.0.40.8` auf extern `cloud.tbz.ch`, angepasst werden.


In [ ]:
%%bash
sed -i -e 's/10.0.40.8/cloud.tbz.ch/g' iiot/mqtt-device-ui/mqtt-device-ui.yaml

Dann einen laufenden MQTT-Operator

In [ ]:
%%bash
cd iiot
kubectl label namespace default istio-injection=enabled
kubectl apply  -f mqtt-operator
kubectl wait --for=condition=Ready pod mqtt-operator --timeout=300s
kubectl get pods

Jetzt können wir mit der Erstelllung der Sensoren beginnen.

Wir brauchen ein Sensor mit drei Buttons
* invoicing
* shipment
* order

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: iiot.mc-b.ch/v1alpha1
kind: Sensor
metadata:
  name: invoicing
spec:
  name: invoicing
  type: Button
  topic: invoicing
  values:
    - name: invoicing 
      unit: "push"
      value: "0.0"
---
apiVersion: iiot.mc-b.ch/v1alpha1
kind: Sensor
metadata:
  name: shipment
spec:
  name: shipment
  type: Button
  topic: shipment
  values:
    - name: shipment
      unit: "push"
      value: "0.0"
---      
apiVersion: iiot.mc-b.ch/v1alpha1
kind: Sensor
metadata:
  name: order
spec:
  name: order
  type: Button
  topic: order
  values:
    - name: order
      unit: "push"
      value: "0.0"      
EOF


Dann braucht es die Deklaration des Gerätes

In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: iiot.mc-b.ch/v1alpha1
kind: Device
metadata:
  name: touchscreen
spec:
  type: Touch Screen
  topic: touchscreen
  sensors:
    - name: invoicing
      sensorRef: invoicing
    - name: shipment
      sensorRef: shipment      
    - name: order
      sensorRef: order      
EOF


Und für die Kommunikation mittels MQTT und Umwandlung der Protokolle/Daten die effektive Implementierung


In [ ]:
%%bash
kubectl apply -f - <<EOF
apiVersion: iiot.mc-b.ch/v1alpha1
kind: MQTTDevice
metadata:
  name: $(cat ~/work/server-ip)-ch
spec:
  deviceRef: touchscreen  # Name der referenzierten Device-Ressource
  mqttSettings:
    broker: mqtt://cloud.tbz.ch:11883  # URL des MQTT-Brokers
    topic: $(cat ~/work/server-ip)-ch  # MQTT-Topic für das Gerät
    qos: 1  # Quality of Service Level (0, 1 oder 2)
    retain: false  # Nachricht zurückhalten (true oder false)
  storage:
    pvcName: "data-claim"
    mountPath: "/data"
---
apiVersion: iiot.mc-b.ch/v1alpha1
kind: MQTTDevice
metadata:
  name: $(cat ~/work/server-ip)-at
spec:
  deviceRef: touchscreen  # Name der referenzierten Device-Ressource
  mqttSettings:
    broker: mqtt://cloud.tbz.ch:11883  # URL des MQTT-Brokers
    topic: $(cat ~/work/server-ip)-at  # MQTT-Topic für das Gerät
    qos: 1  # Quality of Service Level (0, 1 oder 2)
    retain: false  # Nachricht zurückhalten (true oder false)
  storage:
    pvcName: "data-claim"
    mountPath: "/data"
---
apiVersion: iiot.mc-b.ch/v1alpha1
kind: MQTTDevice
metadata:
  name: $(cat ~/work/server-ip)-d
spec:
  deviceRef: touchscreen  # Name der referenzierten Device-Ressource
  mqttSettings:
    broker: mqtt://cloud.tbz.ch:11883  # URL des MQTT-Brokers
    topic: $(cat ~/work/server-ip)-d  # MQTT-Topic für das Gerät
    qos: 1  # Quality of Service Level (0, 1 oder 2)
    retain: false  # Nachricht zurückhalten (true oder false)
  storage:
    pvcName: "data-claim"
    mountPath: "/data"
EOF


Das User Interface liefert uns eine Übersicht über das erstelle Gerät

In [ ]:
%%bash
cd iiot
kubectl apply -f mqtt-device-ui    
echo "http://"$(cat ~/work/server-ip)":$(kubectl get service mqtt-device-ui -o=jsonpath='{ .spec.ports[0].nodePort }')/" 

In [ ]:
%%bash
for dach in ch at d
do
    kubectl logs mqtt-listener-$(cat ~/work/server-ip)-${dach}
done

## Broker und Webshop (Empfänger) 

An folgenden Broker und Topcis wollen wir senden

In [ ]:
%%bash
export NS_BRKR='ms-brkr'
echo "Broker URL (intern)"
kubectl get broker default -o jsonpath='{.status.address.url}' --namespace ${NS_BRKR}
echo ""
echo "------------------------"
echo "Broker URL (extern)"
echo "http://$(cat ~/work/server-ip):"$(kubectl get svc broker-ingress -n knative-eventing -o jsonpath='{.spec.ports[?(@.port==80)].nodePort}')/${NS_BRKR}/default
echo "------------------------"
echo "Triggers"
kn trigger list --namespace ${NS_BRKR}
echo "------------------------"
echo "Microservice Menu"
echo "http://"$(cat ~/work/server-ip)":"$(kubectl get service --namespace ${NS_BRKR} webshop -o=jsonpath='{ .spec.ports[0].nodePort }')/webshop

Im Log des Brokers sehen wir die Daten oder evtl. Fehler

In [ ]:
%%bash
kubectl logs -n knative-eventing deploy/mt-broker-ingress

---

### Aufräumen

In [ ]:
%%bash
kubectl delete mqttdevice $(cat ~/work/server-ip)-ch
kubectl delete mqttdevice $(cat ~/work/server-ip)-at
kubectl delete mqttdevice $(cat ~/work/server-ip)-d
kubectl delete device touchscreen
kubectl delete sensor invoicing
kubectl delete sensor shipment
kubectl delete sensor order
cd iiot
kubectl delete -f mqtt-operator # --grace-period=0 --force 
kubectl delete -f mqtt-device-ui # --grace-period=0 --force 
kubectl delete -f crd
